# Hackathon 8

Written by Eleanor Quint

Topics:
- Eager Execution
- OpenAI Gym
- Deep Q Networks
    - Replay memory
    - Policy and target networks
    - Gradient clipping

This is all setup in a IPython notebook so you can run any code you want to experiment with. Feel free to edit any cell, or add some to run your own code.

Thanks to [OpenAI Baselines](https://github.com/openai/baselines) from which I've learned a lot about RL code. I reccomend starting there if you're doing any RL project.

In [1]:
# We'll start with our library imports...
from __future__ import print_function

import collections
import math
import os
import random

import numpy as np
import tensorflow as tf

import atari_wrappers           # from OpenAI Baselines
import gym                      # for the RL environments
import matplotlib.pyplot as plt # for plots

tf.enable_eager_execution()
print(tf.executing_eagerly())

True


You might notice that we're loading many more libraries than usual. This is because, even for a minimal implementation of DQN and the RL learning problem, it's helpful to import a lot of functionality.

Further, this hackathon will be using TensorFlow's [eager execution mode](https://www.tensorflow.org/guide/eager). This means that, rather than separating graph construction and execution, they are implicitly mixed. Eager mode does not use a Session, nor does it explicitly initialize Variables. Instead, Variables are treated like Python objects and TF function calls return their computed output immediately. This allows us to mix Python control flow with TensorFlow operations.

The first thing we'll do, rather than loading data, is to create a learning environment. We're using some functions from `atari_wrappers.py` for this (because OpenAI writes really good code). This creates a [Gym environment](https://gym.openai.com/docs/) which provides a standard API for us to use. We could use one of a long list of [environments provided by Gym](https://gym.openai.com/envs/), but we'll start with the game that started it all, [Breakout](https://www.youtube.com/watch?v=TmPfTpjtdgg).

In [4]:
env = atari_wrappers.wrap_deepmind(atari_wrappers.make_atari('BreakoutNoFrameskip-v4'), frame_stack=True)
NUM_ACTIONS = env.action_space.n
OBS_SHAPE = env.observation_space.shape

(84, 84, 4)

Environments have two main functions that we'll use: `env.reset()` and `env.step()`. 

`reset()` is used to start an episode and returns an initial observation
```
observation = env.reset()
```

`step()` is used to advance the environment by passing an action to be taken by the agent
```
observation, reward, done, info = env.step(action)
```
Each step returns a tuple of four values: the observation of the environment state (in our case, an image) that results from taking the action, the reward derived from taking the action at the previous state (usually a float), a boolean which indicates whether the episode has finished, and a dict with any extra information. We can get more specific information about the actions and observations with `env.action_space` and `env.observation_space`.

This is an implementation of the classic “agent-environment loop”. Each timestep, the agent chooses an action, and the environment returns an observation and a reward.

Next, we'll setup the replay memory

In [ ]:
Transition = collections.namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

The replay memory just stores $(s,a,r,s')$ transitions for us so we can sample them out of order. This is very important for DQN because, without de-correlating transitions by sampling them out of order, DQN's q-value predictions will almost always diverge. This also allows us to sample large training batches in the online RL setting (i.e., when we're training as we're gathering experiences)

Then, to specify the network itself, we'll use [tf.keras](https://www.tensorflow.org/guide/keras), a library built-in to TensorFlow which provides convenience functions. We'll set up a Keras [Sequential model](https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential) because we're specifying a simple network, which acts as a python callable.

In [ ]:
def dqn_network(input_shape, action_num):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(16, 5, 2, input_shape=input_shape),
        tf.keras.layers.Conv2D(32, 5, 2),
        tf.keras.layers.Conv2D(32, 5, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(action_num)
    ])

Next, we need to write a function which controls the exploration process. We'll use a method called [epsilon-greedy exploration](http://home.deib.polimi.it/restelli/MyWebSite/pdf/rl5.pdf) where we explore (take a random action) with probability $\epsilon$ and exploit (choose the greedy action from the policy network) with probability $1-\epsilon$. We initially set $\epsilon$ to a large value to explore a lot, and then decay it over training to a small value.

In [ ]:
EPS_START = 1.
EPS_END = 0.1
EPS_DECAY = 100000 # number of over which to decay EPS, i.e., after n steps, EPS == EPS_END

def select_eps_greedy_action(policy_model, obs, step, num_actions):
    """
    Args:
        policy_model (callable): mapping of `obs` to q-values
        obs (np.array): current state observation
        step (int): training step count
        num_actions (int): number of actions available to the agent
    """
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * step / EPS_DECAY)
    
    if random.random() > eps_threshold: # exploit
        action = tf.argmax(policy_model(tf.convert_to_tensor(obs)), axis=1)
    else: # explore
        action = random.randrange(num_actions)
    return action

Then, we'll use the [Huber loss](https://en.wikipedia.org/wiki/Huber_loss) on the TD-error to calculate gradients. It has quadratic behavior below $\delta$ near 0, and linear behavior above $\delta$. This allows the error to grow quickly as it moves away from zero, but not blow up too quickly.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cc/Huber_loss.svg/1920px-Huber_loss.svg.png" width="50%">

We'll use [tf.where](https://www.tensorflow.org/api_docs/python/tf/where) to implement the piecewise function.

In [ ]:
def huber_loss(x, delta=1.0):
    """Reference: https://en.wikipedia.org/wiki/Huber_loss"""
    return tf.where(
        tf.abs(x) < delta,
        tf.square(x) * 0.5,
        delta * (tf.abs(x) - 0.5 * delta)
    )

The function which calculates gradients is long, but not too complex. The first step is to sample a batch of transitions from the replay memory and rearrange them so they can be used easily. Then, we'll use a feature of Eager execution called the [GradientTape](https://www.tensorflow.org/tutorials/eager/automatic_differentiation), which allows us to calculate gradients without a fixed graph structure. Within that scope, we'll calculate the [TD-error](https://en.wikipedia.org/wiki/Temporal_difference_learning) as the difference between Q-value output by the policy network and the Q-value calculated with one-step lookahead, $Q(s_t,a) = r_t + \gamma \max_a Q(s_{t+1},a)$. We'll calculate the Huber loss, and then get the gradients of the policy network variables from [tf.GradientTape.gradient](https://www.tensorflow.org/api_docs/python/tf/GradientTape#gradient). Finally, gradients are clipped to help ensure stability of the Q-value estimates.

In [ ]:
def dqn_gradients(replay_memory, policy_model, target_model, batch_size, gamma=0.99, grad_norm_clipping=1.0):
    # before enough transitions are collected to form a batch
    if len(replay_memory) < batch_size:
        return None, None

    # prepare training batch
    transitions = replay_memory.sample(batch_size)
    batch = Transition(*zip(*transitions))
    next_states = np.array(batch.next_state, dtype=np.float32)
    state_batch = np.array(batch.state, dtype=np.float32)
    action_batch = np.array(batch.action, dtype=np.int64)
    reward_batch = np.array(batch.reward)

    with tf.GradientTape() as tape:
        # calculate value from taking action
        action_idxs = np.stack([np.arange(batch_size, dtype=np.int32), action_batch], axis=1)
        state_action_values = tf.gather_nd(policy_model(state_batch), action_idxs)
        
        # calculate best value at next state
        next_state_values = tf.reduce_max(target_model(next_states), axis=1)

        # compute the expected Q values
        expected_state_action_values = (next_state_values * gamma) + reward_batch

        # compute Huber loss on TD-error
        td_error = state_action_values - expected_state_action_values
        loss = huber_loss(td_error)
        gradients = tape.gradient(loss, policy_model.trainable_variables)

    # clip gradients
    for i, grad in enumerate(gradients):
        if grad is not None:
            gradients[i] = tf.clip_by_norm(grad, grad_norm_clipping)
    return loss, gradients

Finally, we'll actually perform training. We setup the models and then, at each step, collect one transition with the current policy network and then train with one batch from the replay memory. At some interval we'll update the target network to the most recent parameters of the policy network with `tf.keras.model.Sequential`'s `get_weights` and `set_weights` functions.

In [ ]:
TARGET_UPDATE_STEP_FREQ = 10
BATCH_SIZE = 64
EPISODE_NUM = 20
REPLAY_BUFFER_SIZE = 100000
LEARNING_RATE = 1e-3

# setup models, replay memory, and optimizer
policy_model = dqn_network(OBS_SHAPE, NUM_ACTIONS)
target_model = dqn_network(OBS_SHAPE, NUM_ACTIONS)
replay_memory = ReplayMemory(REPLAY_BUFFER_SIZE)
optimizer = tf.train.RMSPropOptimizer(LEARNING_RATE)
    
step = 0
for episode in range(EPISODE_NUM):
    # initialize environment
    prev_observation = env.reset()
    observation, reward, done, _ = env.step(random.randrange(NUM_ACTIONS))
    done = False
    ep_score = 0.

    while not done: # until the episode ends
        # select and perform an action
        prepped_obs = np.expand_dims(np.array(observation, dtype=np.float32), axis=0)
        action = select_eps_greedy_action(policy_model, prepped_obs, step, NUM_ACTIONS)
        observation, reward, done, _ = env.step(action)
        # add to memory
        replay_memory.push(prev_observation, action, observation, reward)
        prev_observation = observation

        # train model
        loss, grads = dqn_gradients(replay_memory, policy_model, target_model, BATCH_SIZE)
        if grads is not None:
            optimizer.apply_gradients(zip(grads, policy_model.trainable_variables))
        # increment counters
        ep_score += reward
        step += 1

    # update the target network, copying all variables in DQN
    if episode % TARGET_UPDATE_STEP_FREQ == 0:
        target_model.set_weights(policy_model.get_weights())
    print("Episode {} achieved score {} at {} training steps".format(episode, ep_score, step))

And that's basically how we implement [Mnih et al.'s Deep Q Network](https://www.nature.com/articles/nature14236)!

## No Hackathon due this week.
Enjoy life, or work on your project or something.